##  Pytorch Dataloader 效率问题
pytorch 没有比较高效的数据存储, 例如读取图片，需要cv.imread，利用opencv来读取，非常耗时。
读取txt文件也是如此，比如将点云数据存储为文本格式，读取时间同样也是非常耗时的。
这里可以利用lmdb，h5py,pth,lmdb,n5

当时看到了一个还不错的github链接：



https://github.com/Lyken17/Efficient-PyTorch



主要是讲如何使用lmdb，h5py,pth,lmdb,n5等数据存储方式皆可以。



个人的感受是，h5在数据调用上比较快，但是如果要使用多线程读写，就尽量不要使用h5,因为h5的多线程读写好像比较麻烦。



http://docs.h5py.org/en/stable/mpi.html



这里贴一下h5数据的读写代码(主要需要注意的是字符串的读写需要encode,decode,最好用create_dataset，直接写的话读的时候会报错)：



写：
```python
    imagenametotal_.append(os.path.join('images', imagenametotal).encode())
    with h5py.File(outfile) as f:
        f.create_dataset('imagename', data=imagenametotal_)
        f['part'] = parts_
        f['S'] = Ss_
        f['image'] = cvimgs
```

读：

```python

with h5py.File(outfile) as f:
    imagename = [x.decode() for x in f['imagename']]
    kp2ds = np.array(f['part'])
    kp3ds = np.array(f['S'])
    cvimgs = np.array(f['image'])
```

## 多loss 函数问题
这个好像没有遇到过。

当loss函数有多个组成的时候，比如 loss = loss1 + loss2 + loss3



那么需要把这三个loss写到一个class中，然后再forward里面将其加起来。

## gpu利用率不高+gpu现存占用浪费



常用配置：



（1）主函数前面加：（这个会牺牲一点点显存提高模型精度）


```python
cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.enabled = True
```

（2）训练时，epoch前面加：（定期清空模型，效果感觉不明显）



`torch.cuda.empty_cache()`


（3）无用变量前面加：（同上，效果某些操作上还挺明显的）



`del xxx(变量名)`


（4）dataloader的长度_len_设置：（dataloader会间歇式出现卡顿，设置成这样会避免不少）


```python
def __len__(self):
    return self.images.shape[0]
```

（5）dataloader的预加载设置：（会在模型训练的时候加载数据，提高一点点gpu利用率）


```python
train_loader = torch.utils.data.DataLoader(
        train_dataset,
        pin_memory=True,
    )
```

（6）网络设计很重要，外加不要初始化任何用不到的变量，因为pyroch的初始化和forward是分开的，他不会因为你不去使用，而不去初始化。

